In [ ]:
# Some Preliminary work using the NaiveBayes and Random Forest.

In [238]:
import pandas as pd
import os
from nltk import word_tokenize

In [239]:
# Import data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [240]:
# Check Data Stats
display(train.head())
print(f"Data Size: {train.shape}")
train['is_duplicate'].value_counts()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


Data Size: (404290, 6)


0    255027
1    149263
Name: is_duplicate, dtype: int64

In [241]:
train.dropna(inplace=True)

## simple TFIDF + NaiveBayes

In [275]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
tfidf = TfidfVectorizer(tokenizer=word_tokenize, stop_words='english')

In [276]:
q_full = pd.concat([train['question1'],train['question2']])

In [244]:
# Fit Vectorizer with full text space
tfidf.fit(list(q_full))

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function word_tokenize at 0x156c301e0>, use_idf=True,
        vocabulary=None)

In [245]:
# Vectorize
q1_tfidf = tfidf.transform(list(train['question1']))
q2_tfidf = tfidf.transform(list(train['question2']))

In [246]:
# Kind of representing senatic overlap with addition
import numpy as np
q_diff = np.add(q1_tfidf,q2_tfidf)

In [248]:
ans = train['is_duplicate']

In [249]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(q_diff, ans, test_size = 0.25)

In [250]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train, y_train)
print(classification_report(y_test, nb.predict(X_test)))
print(f'AUC: {roc_auc_score(y_test, nb.predict(X_test))}')
print(f'Accuracy: {accuracy_score(y_test, nb.predict(X_test))}')

0.7459929555168593

In [251]:
from sklearn.metrics import classification_report,roc_auc_score, accuracy_score
print(classification_report(y_test, nb.predict(X_test)))
print(f'AUC: {roc_auc_score(y_test, nb.predict(X_test))}')
print(f'Accuracy: {accuracy_score(y_test, nb.predict(X_test))}')

             precision    recall  f1-score   support

          0       0.77      0.86      0.81     63740
          1       0.70      0.55      0.62     37332

avg / total       0.74      0.75      0.74    101072

AUC: 0.7060603141021677
Accuracy: 0.7459929555168593


In [252]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
print(classification_report(y_test, rf.predict(X_test)))
print(f'AUC: {roc_auc_score(y_test, rf.predict(X_test))}')
print(f'Accuracy: {accuracy_score(y_test, rf.predict(X_test))}')

0.7946216558492956

In [253]:
print(classification_report(y_test, rf.predict(X_test)))
print(f'AUC: {roc_auc_score(y_test, rf.predict(X_test))}')
print(f'Accuracy: {accuracy_score(y_test, rf.predict(X_test))}')

             precision    recall  f1-score   support

          0       0.79      0.91      0.85     63740
          1       0.80      0.60      0.68     37332

avg / total       0.79      0.79      0.79    101072

AUC: 0.7534493381935634
Accuracy: 0.7946216558492956


In [254]:
from sklearn.metrics import log_loss

In [255]:
log_loss(y_test, rf.predict(X_test))

7.09356156920396

## Stopwords Removed TFIDF + NaiveBayes

In [277]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
tfidf = TfidfVectorizer(tokenizer=word_tokenize, stop_words='english')

In [278]:
# Fit Vectorizer with full text space
tfidf.fit(list(q_full))

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function word_tokenize at 0x156c301e0>, use_idf=True,
        vocabulary=None)

In [279]:
# Vectorize
q1_tfidf = tfidf.transform(list(train['question1']))
q2_tfidf = tfidf.transform(list(train['question2']))

In [280]:
q_diff = np.add(q1_tfidf,q2_tfidf)

In [281]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(q_diff, ans, test_size = 0.25)

In [282]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train, y_train)
print(classification_report(y_test, nb.predict(X_test)))
print(f'AUC: {roc_auc_score(y_test, nb.predict(X_test))}')
print(f'Accuracy: {accuracy_score(y_test, nb.predict(X_test))}')

             precision    recall  f1-score   support

          0       0.77      0.85      0.81     63780
          1       0.69      0.56      0.62     37292

avg / total       0.74      0.74      0.74    101072

AUC: 0.7046363637984225
Accuracy: 0.7434304258350483


In [283]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
print(classification_report(y_test, rf.predict(X_test)))
print(f'AUC: {roc_auc_score(y_test, rf.predict(X_test))}')
print(f'Accuracy: {accuracy_score(y_test, rf.predict(X_test))}')

             precision    recall  f1-score   support

          0       0.81      0.90      0.85     63780
          1       0.79      0.63      0.70     37292

avg / total       0.80      0.80      0.80    101072

AUC: 0.7650476394255472
Accuracy: 0.8014583663131233


In [284]:
log_loss(y_test, rf.predict(X_test))

6.857433538975535

## Fine-tuning TFIDF

In [290]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
tfidf = TfidfVectorizer(tokenizer=word_tokenize, stop_words='english', max_df = 0.8, min_df=2)

In [291]:
# Fit Vectorizer with full text space
tfidf.fit(list(q_full))
# Vectorize
q1_tfidf = tfidf.transform(list(train['question1']))
q2_tfidf = tfidf.transform(list(train['question2']))

q_diff = np.add(q1_tfidf,q2_tfidf)
print(q_diff.shape)

(404287, 59123)


In [293]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(q_diff, ans, test_size = 0.25)

In [294]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train, y_train)
print(classification_report(y_test, nb.predict(X_test)))
print(f'AUC: {roc_auc_score(y_test, nb.predict(X_test))}')
print(f'Accuracy: {accuracy_score(y_test, nb.predict(X_test))}')

             precision    recall  f1-score   support

          0       0.77      0.85      0.81     63836
          1       0.68      0.57      0.62     37236

avg / total       0.74      0.74      0.74    101072

AUC: 0.7083300501996027
Accuracy: 0.7447463194554377


In [295]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
print(classification_report(y_test, rf.predict(X_test)))
print(f'AUC: {roc_auc_score(y_test, rf.predict(X_test))}')
print(f'Accuracy: {accuracy_score(y_test, rf.predict(X_test))}')

             precision    recall  f1-score   support

          0       0.81      0.90      0.85     63836
          1       0.78      0.64      0.70     37236

avg / total       0.80      0.80      0.80    101072

AUC: 0.768286608938574
Accuracy: 0.8018442298559443


In [296]:
log_loss(y_test, rf.predict(X_test))

6.844110461447055

## With Stemming

In [365]:
from nltk.stem.snowball import SnowballStemmer 
snowball = SnowballStemmer('english')
stemmer = snowball.stemmer.stem

In [366]:
def stem_tokenizer(sent):
    tokens = word_tokenize(sent)
    tokens = [stemmer(token) for token in tokens]
    return tokens

In [367]:
tfidf = TfidfVectorizer(tokenizer=stem_tokenizer, stop_words='english', max_df = 0.8, min_df=2)

In [368]:
# Fit Vectorizer with full text space
tfidf.fit(list(q_full))
# Vectorize
q1_tfidf = tfidf.transform(list(train['question1']))
q2_tfidf = tfidf.transform(list(train['question2']))

q_diff = np.add(q1_tfidf,q2_tfidf)
print(q_diff.shape)

(404287, 45149)


In [369]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(q_diff, ans, test_size = 0.25)

In [370]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train, y_train)
print(classification_report(y_test, nb.predict(X_test)))
print(f'AUC: {roc_auc_score(y_test, nb.predict(X_test))}')
print(f'Accuracy: {accuracy_score(y_test, nb.predict(X_test))}')

             precision    recall  f1-score   support

          0       0.76      0.84      0.80     63650
          1       0.68      0.56      0.61     37422

avg / total       0.73      0.74      0.73    101072

AUC: 0.7004448089417976
Accuracy: 0.7379392908025961


In [371]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
print(classification_report(y_test, rf.predict(X_test)))
print(f'AUC: {roc_auc_score(y_test, rf.predict(X_test))}')
print(f'Accuracy: {accuracy_score(y_test, rf.predict(X_test))}')

             precision    recall  f1-score   support

          0       0.81      0.90      0.85     63650
          1       0.79      0.64      0.70     37422

avg / total       0.80      0.80      0.80    101072

AUC: 0.7681957267660331
Accuracy: 0.8018442298559443
